# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import yfinance as yf

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataprojectyf as dp


# Read and clean data

Import your data, either through an API or manually, and load it. 

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [19]:
# a. fetch the data for the house representatives
house_raw = dp.fetch_data(mode="house", print_df = True)

# b. fech example data from yfinance
example_yf = yf.download("AAPL", "2022-03-01", "2022-03-31", interval="1d")

display(example_yf)



request successful


,date,ticker,amount,action,representative,party,description
0,2021-09-27,BP,"$1,001 - $15,000",purchase,Virginia Foxx,Republican,BP plc
1,2021-09-13,XOM,"$1,001 - $15,000",purchase,Virginia Foxx,Republican,Exxon Mobil Corporation
2,2021-09-10,ILPT,"$15,001 - $50,000",purchase,Virginia Foxx,Republican,Industrial Logistics Properties Trust - Common...
3,2021-09-28,PM,"$15,001 - $50,000",purchase,Virginia Foxx,Republican,Phillip Morris International Inc
4,2021-09-17,BLK,"$1,001 - $15,000",sale_partial,Alan S. Lowenthal,Democrat,BlackRock Inc
...,...,...,...,...,...,...,...
16943,2020-04-09,SWK,"$1,001 - $15,000",sale_partial,Ed Perlmutter,Democrat,"Stanley Black & Decker, Inc."
16944,2020-04-09,USB,"$1,001 - $15,000",sale_partial,Ed Perlmutter,Democrat,U.S. Bancorp
16945,2020-03-13,BMY,"$100,001 - $250,000",sale_full,Van Taylor,Republican,Bristol-Myers Squibb Company
16946,2020-03-13,LLY,"$500,001 - $1,000,000",sale_full,Van Taylor,Republican,Eli Lilly and Company


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-01,164.699997,166.600006,161.970001,163.199997,162.217346,83474400
2022-03-02,164.389999,167.360001,162.949997,166.559998,165.557098,79724800
2022-03-03,168.470001,168.910004,165.550003,166.229996,165.229080,76678400
2022-03-04,164.490005,165.550003,162.100006,163.169998,162.187515,83737200
2022-03-07,163.360001,165.020004,159.039993,159.300003,158.340836,96418800
2022-03-08,158.820007,162.880005,155.800003,157.440002,156.492020,131148300
2022-03-09,161.479996,163.410004,159.410004,162.949997,161.968857,91454900
2022-03-10,160.199997,160.389999,155.979996,158.520004,157.565521,105342000
2022-03-11,158.929993,159.279999,154.500000,154.729996,153.798340,96970100


# Merge data sets

In [ ]:
# b. clean the data
house_clean = dp.clean_data(house_raw, print_df = True)

In [7]:
house_clean = dp.average_amount(house_clean)
nancy = dp.select_rep(house_clean, "Nancy Pelosi", print_df = True)


,date,ticker,action,representative,party,description,min_amount,max_amount,avg_amount
0,2020-08-07,FB,sale_full,Nancy Pelosi,dem,"Facebook, Inc. - Class A",1000001.0,5000000.0,3000000.5
1,2021-03-10,RBLX,purchase,Nancy Pelosi,dem,Roblox Corporation - purchase 10K shares,500001.0,1000000.0,750000.5
2,2021-02-18,AB,purchase,Nancy Pelosi,dem,"AllianceBerstein Holding LP. Purchased 15,000 ...",500001.0,1000000.0,750000.5
3,2021-02-23,AB,purchase,Nancy Pelosi,dem,"AllianceBerstein Holding LP. Purchased 25,000 ...",500001.0,1000000.0,750000.5
4,2020-06-24,AXP,purchase,Nancy Pelosi,dem,American Express Company,100001.0,250000.0,175000.5
5,2020-06-18,AAPL,sale_full,Nancy Pelosi,dem,Apple Inc.,1000001.0,5000000.0,3000000.5
6,2020-06-18,NFLX,purchase,Nancy Pelosi,dem,"Netflix, Inc.",1000001.0,5000000.0,3000000.5
7,2020-06-12,PYPL,purchase,Nancy Pelosi,dem,"PayPal Holdings, Inc.",500001.0,1000000.0,750000.5
8,2020-06-12,PYPL,purchase,Nancy Pelosi,dem,"PayPal Holdings, Inc.",500001.0,1000000.0,750000.5
9,2020-06-24,PYPL,purchase,Nancy Pelosi,dem,"PayPal Holdings, Inc.",250001.0,500000.0,375000.5


Explain what you see when moving elements of the interactive plot around. 

In [16]:
stocks_price = dp.get_stock_data(nancy)

[*********************100%***********************]  22 of 22 completed

2 Failed downloads:
- FB: No timezone found, symbol may be delisted
- WORK: No timezone found, symbol may be delisted


In [17]:
display(stocks_price)

Adj Close                                                 \
                  AAPL         AB        AMZN         AXP         CRM   
Date                                                                    
2020-02-20   78.513992  27.180937  107.654999  130.456650  193.360001   
2020-02-21   76.736816  26.504711  104.798500  128.851990  189.500000   
2020-02-24   73.091782  25.665266  100.464500  122.442818  185.940002   
2020-02-25   70.616013  25.144499   98.637001  115.479645  181.270004   
2020-02-26   71.736229  24.911320   98.979500  113.187256  178.869995   
...                ...        ...         ...         ...         ...   
2022-12-22  132.028412  34.223328   83.790001  144.273575  129.190002   
2022-12-23  131.658981  33.682755   85.250000  145.971390  129.440002   
2022-12-27  129.831772  33.496014   83.040001  145.345886  130.660004   
2022-12-28  125.847855  32.562290   81.820000  142.982864  128.470001   
2022-12-29  129.412415  33.771214   84.180000  146.269257  132.539993   

                                                              ...    Volume  \
                  CRWD         DIS  FB       GOOG      GOOGL  ...      MSFT   
Date                                                          ...             
2020-02-20   63.380001  140.369995 NaN  75.907501  75.849503  ...  36862400   
2020-02-21   60.930000  138.970001 NaN  74.255501  74.172997  ...  48572600   
2020-02-24   58.080002  133.009995 NaN  71.079498  70.992996  ...  68311100   
2020-02-25   57.740002  128.190002 NaN  69.422501  69.316002  ...  68073300   
2020-02-26   58.459999  123.360001 NaN  69.658997  69.523499  ...  56206100   
...                ...         ...  ..        ...        ...  ...       ...   
2022-12-22  103.970001   86.669998 NaN  88.260002  87.760002  ...  28651700   
2022-12-23  101.989998   88.010002 NaN  89.809998  89.230003  ...  21207000   
2022-12-27  100.660004   86.370003 NaN  87.930000  87.389999  ...  16688600   
2022-12-28   99.959999   84.169998 NaN  86.459999  86.019997  ...  17457100   
2022-12-29  104.570000   87.180000 NaN  88.949997  88.449997  ...  19770700   

                                                                           \
                  MU     NFLX       NVDA      PYPL        RBLX       TSLA   
Date                                                                        
2020-02-20  20797100  4079400   81005200   7212900         NaN  264523500   
2020-02-21  27038900  3930100   76818000   5894500         NaN  214722000   
2020-02-24  33321100  6936400   85691600  10139300         NaN  227883000   
2020-02-25  41894200  6481200  105549600  13636100         NaN  259357500   
2020-02-26  28075600  8934100   74773200  10301800         NaN  211282500   
...              ...      ...        ...       ...         ...        ...   
2022-12-22  40959600  7856200   56504500  16488200  12842900.0  210090300   
2022-12-23  17425700  4251100   34932600   9990400  10774500.0  166989700   
2022-12-27  15159100  5778100   46490200  10323800  11084000.0  208643400   
2022-12-28  12518700  5964400   35106600   8897600   9948900.0  221070500   
2022-12-29  13204300  9588500   35492300  13143000  11301200.0  221923300   

                                     
                   V       WBD WORK  
Date                                 
2020-02-20   8531900   5705100  NaN  
2020-02-21   9231500   4574600  NaN  
2020-02-24  13316800   7157200  NaN  
2020-02-25  18539000   8349800  NaN  
2020-02-26  14214700   6230400  NaN  
...              ...       ...  ...  
2022-12-22   5690300  28566000  NaN  
2022-12-23   3246000  19035100  NaN  
2022-12-27   2904900  22207700  NaN  
2022-12-28   3139200  22506100  NaN  
2022-12-29   3675500  29740700  NaN  

[722 rows x 132 columns]

# Analysis

# Conclusion